### Elementy sztucznej inteligencji
# Projekt: Binarne Drzewa Decyzyjne
autorzy:
1. Arkadiusz Florek
2. Maciej Komosa
3. Albert Pieniądz
4. Jakub Zięba

## TODO: Id3, opis jak działa wezel
### błędy:
1. dane nie są formatowane jak dane w przykładzie w linku (na dole)
2. prawdopodobniej ostatecznie trzeba będzie usunąć klasę, bo trzeba było to zrobic w skrypcie, ale w klasie lepiej widać co jest grane
### możliwe rozwiązania: zastosować formatowanie z maila od prof'a albo jakieś prostsze

In [40]:
import numpy as np, pandas as pd

Funkcje do obliczania entropii dla danej konkluzji

In [41]:
from math import log2

class Wezel:

    def __init__(self):
        self.wartosc = None
        self.kolejny = None
        self.dzieci = None

class DrzewoBinarneID3:

    def __init__(self, dane):
        #self.dane to nie są dane, w obrębie klasy korzystamy tylko z self.danych
        self.dane = dane[1:, :]
        #nagłówki
        self.nazwy_przeslanek = dane[0, :]
        #konkluzje z danych
        self.konkluzje = dane[1:, -1]
        #self.explanatory hehe
        self.klasy_konkluzji = list(set(self.dane[:, -1]))
        #jestem prawie pewny że połowa z tych rzeczy jest niepotrzebna
        self.ilosc_wierszy_dla_danej_klasy_konkluzji = [list(self.dane[:, -1]).count(x) for x in self.klasy_konkluzji]
        #entropia całego układu, czyli właściwie to klas
        self.entropia = self._entropia(self.dane[:, -1])
        #teoretycznie w tym węźle ma się mieścić całe drzewo, na zasadzie zagnieżdżonych referencji
        #byłoby jeszcze zabawniej jakbym rozumiał jak to zrobić 
        self.wezel = Wezel();
 
    def _entropia(self, dane):
        # Tworzymy słownik, w którym kluczem jest wartość, a wartością liczba wystąpień tej wartości
        wystapienia_wartosci = {}
        for wiersz in dane:
            atrybut = wiersz[-1]
            if atrybut not in wystapienia_wartosci:
                wystapienia_wartosci[atrybut] = 0
            wystapienia_wartosci[atrybut] += 1
    
        # Obliczamy entropię dla danej tabeli
        entropia = 0.0
        ilosc_wierszy = len(dane)
        for wartosc in wystapienia_wartosci.values():
            prawdopodobienstwo = wartosc / ilosc_wierszy
            entropia -= prawdopodobienstwo * log2(prawdopodobienstwo)
    
        return entropia
 
    def _zysk_informacji(self, dane_indeksy, przeslanka):
        # Tworzymy słownik, w którym kluczem jest wartość, a wartością lista wierszy z tą wartością
        wartosci_przeslanek = {}
        for wiersz in dane_indeksy:
            wartosc = self.dane[wiersz, przeslanka]
            if wartosc not in wartosci_przeslanek:
                wartosci_przeslanek[wartosc] = []
            wartosci_przeslanek[wartosc].append(self.dane[wiersz])
    
        # Obliczamy entropię dla każdego podziału i sumujemy, aby uzyskać entropię po podziale
        ilosc_wierszy = len(self.dane)
        entropie = []
        for wartosci_przeslanki in wartosci_przeslanek.values():
            ilosc_wartosci_przeslanki = len(wartosci_przeslanki)
            entropia_przeslanki = self._entropia(wartosci_przeslanki)
            entropie.append((ilosc_wartosci_przeslanki / ilosc_wierszy) * entropia_przeslanki)
        entropia_przeslanki = sum(entropie)
    
        # Obliczamy przyrost informacji
        zysk = self._entropia(self.dane) - entropia_przeslanki
    
        return zysk

    def _przeslanka_o_najwiekszym_zysku(self, dane_indeksy, przeslanki_indeksy):
        
        # tabela przeslanek, czyli tabela o wartosciach
        # z zakresu dlugosci jednego wiersza odjąć konkluzję

        entropie_przeslanek = [self._zysk_informacji(dane_indeksy, przeslanka) for przeslanka in przeslanki_indeksy]
        indeks_przeslanka_o_najwiekszym_zysku = przeslanki_indeksy[entropie_przeslanek.index(max(entropie_przeslanek))]

        return self.nazwy_przeslanek[indeks_przeslanka_o_najwiekszym_zysku], indeks_przeslanka_o_najwiekszym_zysku
    
    #szczerze to nie wiem jak ten sposób dzielenia na węzły zrobić, ale ma być w tej funkcji i bazować na funkcji _przeslanka_o_najwiekszym_zysku
    #link: https://towardsdatascience.com/id3-decision-tree-classifier-from-scratch-in-python-b38ef145fd90
    def id3(self):
    
        # indeksy
        dane_indeksy =[*range(len(self.dane))]
        print(dane_indeksy)

        #przeslanki
        przeslanki_indeksy = [*range(len(self.nazwy_przeslanek))]
        print(przeslanki_indeksy)
        self.wezel = self._id3_rekursja(dane_indeksy,
                                   przeslanki_indeksy,
                                   self.wezel)
    
    def _id3_rekursja(self, dane_indeksy, przeslanki_indeksy, wezel):
        if not wezel:
            wezel = Wezel()  # inicjalizacja wezla
        # konkluzje sortowane po indeksach danych
        klasy_w_przeslankach = [self.konkluzje[x] for x in dane_indeksy]
        # jesli wszystkie wiersze maja ta sama konkluzje
        if len(set(klasy_w_przeslankach)) == 1:
            wezel.wartosc = self.konkluzje[dane_indeksy[0]]
            return wezel
        if len(przeslanki_indeksy) == 0:
        # jesli nie ma juz wiecej do policzenia
            wezel.wartosc = max(set(klasy_w_przeslankach), key=klasy_w_przeslankach.count)
            return wezel
        # else...
        # choose the feature that maximizes the information gain
        najlepsza_przeslanka, indeks_najlepszej_przeslanki = self._przeslanka_o_najwiekszym_zysku(dane_indeksy, przeslanki_indeksy)
        wezel.wartosc = najlepsza_przeslanka
        wezel.dzieci = []
        # value of the chosen feature for each instance
        wartosci_przeslanek = list(set([self.dane[i][indeks_najlepszej_przeslanki] for i in dane_indeksy]))
        # loop through all the values
        for wartosc in wartosci_przeslanek:
            dziecko = Wezel()
            dziecko.wartosc = wartosc  # add a branch from the node to each feature value in our feature
            wezel.dzieci.append(dziecko)  # append new child node to current node
            dziecko_dane = [i for i in dane_indeksy if self.dane[i][indeks_najlepszej_przeslanki] == wartosc]
            if not dziecko_dane:
                dziecko.kolejny = max(set([klasy_w_przeslankach]), key=klasy_w_przeslankach.count)
                print('')
            else:
                if przeslanki_indeksy and indeks_najlepszej_przeslanki in przeslanki_indeksy:
                    do_wyciecia = przeslanki_indeksy.index(indeks_najlepszej_przeslanki)
                    przeslanki_indeksy.pop(do_wyciecia)
                # recursively call the algorithm
                dziecko.kolejny = self._id3_rekursja(dziecko_dane, przeslanki_indeksy, dziecko.kolejny)
        return wezel
        

Klasa węzła

In [42]:
#pobieranie danych z nagłówkami i przesyłanie ich do funkcji - dane i nagłówki oddzielnie - trochę śmie
dane = pd.read_csv("sport.csv", header=None).to_numpy()
bdd = DrzewoBinarneID3(dane)
print(bdd.entropia)
print(bdd.id3())
print(bdd.wezel.dzieci)

2.67943596719348
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215]
[0, 1, 2, 3,